# Gotta Process 'Em all!

1. The name, id, base_experience, weight, height and order of all Pokémon that appear in the any of the games red, blue, leafgreen or white.
2. The name of the slot 1 (and if available 2) type of each of the Pokémon's types.
3. The Body Mass Index of the Pokémon (hint: The formula for BMI is weight (kg) /
height (m2))
4. The first letter of names of the Pokémon should be capitalized.
5. The url of the front_default sprite.
6. Prepare the data in an appropriate data format. Consider if it should be multiple or a
single file.

In [9]:
import httpx 
import pprint
import pandas as pd
import asyncio
import time
from urllib.parse import urljoin
from functools import reduce
import pokeapi.helpers as ph

ModuleNotFoundError: No module named 'pokeapi'

In [ ]:
BASE_URL= "https://pokeapi.co/api/v2/"
VERSIONS_OF_INTEREST = ['red', 'blue', 'leafgreen', 'white']

In [ ]:
pokemon_species = []

for vn in VERSIONS_OF_INTEREST:
    with httpx.Client(base_url=BASE_URL) as client:
        version_id, version_group = ph.get_version_id_and_version_group(client=client, version_name=vn)
        version_group_pokedexs = ph.get_version_group_pokedexs(client=client, version_group_name=version_group)
        for pokedex in version_group_pokedexs:
            pokemon_species.append(ph.get_pokemon_species(client=client, pokedex_name=pokedex))

unique_target_pokemon = set(ph.flatten_list(pokemon_species))       


In [ ]:
client = httpx.AsyncClient()
tasks = []
for p in unique_target_pokemon:
    tasks.append(asyncio.ensure_future(ph.get_default_pokemon(client, pokemon_species=p)))
    
pokemon_raw_json = await asyncio.gather(*tasks)
_ = client.aclose()


In [ ]:
raw_df = pd.DataFrame(pokemon_raw_json)
raw_df['slot_1'] = raw_df['types'].str[0].str['type'].str['name']
raw_df['slot_2'] = raw_df['types'].str[1].str['type'].str['name']

raw_df['body_mass_index'] = (raw_df['weight'] / raw_df['height']).round(2)

raw_df['name'] = raw_df['name'].str.title()

raw_df['front_default_sprite_url'] = raw_df['sprites'].str['front_default']

columns_subset = ["name", "id", "base_experience", "weight", "height", "body_mass_index", "order", "slot_1", "slot_2", "front_default_sprite_url"]


output_df = raw_df[columns_subset].sort_values('order').to_csv('data.csv', index=False)

NameError: name 'pd' is not defined